In [209]:
# define the path
GRAMMAR_PATH = "./atis/atis-grammar-cnf.cfg"
SENTENCES_PATH = "./atis/atis-test-sentences.txt"

In [210]:
import nltk

grammar = nltk.data.load(GRAMMAR_PATH)  # load the grammar
raw_sentences = nltk.data.load(SENTENCES_PATH)  # load raw sentences
test_sentences = nltk.parse.util.extract_test_sentences(raw_sentences)  # extract test sentences

In [211]:
## test the parser
# initialize the parser
parser = nltk.parse.BottomUpChartParser(grammar)

#try to parse several test sentences
for sentence in test_sentences[:10]:
    try:
        trees = list(parser.parse(sentence[0]))
        if trees:
            print("Sentence: ", sentence[0], len(trees))
        else:
            print("Parsing failed for sentence: ", sentence[0])
    except Exception as e:
        print("Exception: ", e)
        print("Parsing failed for sentence: ", sentence[0])

Sentence:  ['prices', '.'] 2
Sentence:  ['show', 'availability', '.'] 3
Sentence:  ['show', 'the', 'flights', '.'] 2
Sentence:  ['milwaukee', 'to', 'detroit', '.'] 2
Sentence:  ['indianapolis', 'to', 'seattle', '.'] 2
Sentence:  ['list', 'round', 'trips', '.'] 11
Sentence:  ['list', 'saturday', 'flights', '.'] 5
Parsing failed for sentence:  ['what', 'aircraft', 'is', 'this', '.']
Parsing failed for sentence:  ['list', 'these', 'economy', 'fares', '.']
Exception:  Grammar does not cover some of the input words: "'destinations'".
Parsing failed for sentence:  ['list', 'these', 'city', 'destinations', '.']


In [212]:
def CKY_recognize(sentence, grammar: nltk.grammar.CFG):
    # initialize the chart
    n = len(sentence)
    chart = [[set() for _ in range(n)] for _ in range(n)]

    # fill in the diagonal of the chart
    for i in range(n):
        # get the production whose rhs is the word i
        for production in grammar.productions(rhs=sentence[i]):
            # add the lhs of the production to the cell
            chart[i][i].add(production.lhs())

    # main body of the CKY algorithm
    # traverse the chart for each width b
    for b in range(2, n + 1):
        # for each start position i
        for i in range(0, n - b + 1):
            # for each left width k
            for k in range(0, b - 1):
                # for each non-terminal B and C
                # if there is a production B -> C in the grammar
                # add [B, C] to the cell
                for B in chart[i][i + k]:
                    for C in chart[i + k + 1][i + b - 1]:
                        # nltk grammar productions only accept one item in the rhs
                        for production in grammar.productions(rhs=B):
                            if production.rhs() == (B, C):
                                chart[i][i + b - 1].add(production.lhs())

    # if the start symbol is in chart[0][n-1], the sentence can be parsed
    if grammar.start() in chart[0][n - 1]:
        return True

    return False

In [213]:
# Extra: Optimized running efficiency
def CKY_recognize_optimized(sentence, grammar: nltk.grammar.CFG):
    # initialize the chart
    n = len(sentence)
    chart = [[set() for _ in range(n)] for _ in range(n)]

    # Use production cache: map RHS -> LHS
    # Binary rules (e.g., A -> B C)
    binary_rules = dict()

    # add all binary rules to the cache
    for production in grammar.productions():
        if len(production.rhs()) == 2:
            if production.rhs() not in binary_rules:
                binary_rules[production.rhs()] = []
                binary_rules[production.rhs()].append(production.lhs())
            else:
                binary_rules[production.rhs()].append(production.lhs())

    # fill in the diagonal of the chart
    for i in range(n):
        # get the production whose rhs is the word i
        for production in grammar.productions(rhs=sentence[i]):
            # add the lhs of the production to the cell
            chart[i][i].add(production.lhs())

    # main body of the CKY algorithm
    # traverse the chart for each width b
    for b in range(2, n + 1):
        # for each start position i
        for i in range(0, n - b + 1):
            # for each left width k
            for k in range(0, b - 1):
                # for each non-terminal B and C
                # if there is a production B -> C in the grammar
                # add [B, C] to the cell
                for B in chart[i][i + k]:
                    for C in chart[i + k + 1][i + b - 1]:
                        # look up the cache for the lhs
                        for lhs in binary_rules.get((B, C), []):
                            chart[i][i + b - 1].add(lhs)

    # if the start symbol is in chart[0][n-1], the sentence can be parsed
    if grammar.start() in chart[0][n - 1]:
        return True

    return False

In [214]:
# test the CKY recognizer (running for about 2 minute)
# for i in range(len(test_sentences)):
#     print(i, test_sentences[i][0], end='\t')
#     print(CKY_recognize(test_sentences[i][0], grammar))

In [215]:
# test the optimized CKY recognizer
for i in range(len(test_sentences)):
    print(i, test_sentences[i][0], end='\t')
    print(CKY_recognize_optimized(test_sentences[i][0], grammar))

0 ['prices', '.']	True
1 ['show', 'availability', '.']	True
2 ['show', 'the', 'flights', '.']	True
3 ['milwaukee', 'to', 'detroit', '.']	True
4 ['indianapolis', 'to', 'seattle', '.']	True
5 ['list', 'round', 'trips', '.']	True
6 ['list', 'saturday', 'flights', '.']	True
7 ['what', 'aircraft', 'is', 'this', '.']	False
8 ['list', 'these', 'economy', 'fares', '.']	False
9 ['list', 'these', 'city', 'destinations', '.']	False
10 ['what', 'is', 'the', 'fare', '.']	True
11 ['which', 'flights', 'are', 'cheapest', '.']	False
12 ['list', 'flights', 'from', 'cleveland', '.']	True
13 ['what', 'are', 'the', 'costs', '.']	True
14 ['can', 'i', 'have', 'the', 'fare', '.']	True
15 ['what', 'is', 'e', 'w', 'r', '.']	True
16 ['oakland', 'to', 'salt', 'lake', 'city', '.']	True
17 ['show', 'me', 'northwest', 'flights', 'to', 'detroit', '.']	True
18 ['i', 'want', 'to', 'leave', 'before', 'noon', '.']	True
19 ['what', 'flights', 'leave', 'boston', 'to', 'pittsburgh', '.']	True
20 ['i', "'d", 'like', 'an', 'a

In [216]:
# CKY parser
def CKY_parse(sentence, grammar: nltk.grammar.CFG):
    n = len(sentence)
    chart = [[set() for _ in range(n)] for _ in range(n)]
    # use dicts as back pointers
    back_pointers = [[{} for _ in range(n)] for _ in range(n)]

    binary_rules = dict()

    for production in grammar.productions():
        if len(production.rhs()) == 2:
            if production.rhs() not in binary_rules:
                binary_rules[production.rhs()] = []
                binary_rules[production.rhs()].append(production.lhs())
            else:
                binary_rules[production.rhs()].append(production.lhs())

    for i in range(n):
        for production in grammar.productions(rhs=sentence[i]):
            chart[i][i].add(production.lhs())
            back_pointers[i][i][production.lhs()] = [sentence[i]]

    # main body of the CKY algorithm
    for b in range(2, n + 1):
        for i in range(0, n - b + 1):
            for k in range(0, b - 1):
                for B in chart[i][i + k]:
                    for C in chart[i + k + 1][i + b - 1]:
                        for lhs in binary_rules.get((B, C), []):
                            chart[i][i + b - 1].add(lhs)

                            # add back pointers
                            if lhs not in back_pointers[i][i + b - 1]:
                                back_pointers[i][i + b - 1][lhs] = []
                            back_pointers[i][i + b - 1][lhs].append((B, C, i + k))

    # return back pointers
    if grammar.start() in chart[0][n - 1]:
        return True, back_pointers

    return False, None

In [217]:
from nltk.tree import ImmutableTree


# extract the trees from the back pointers
def extract_trees(back_pointers, i, j, start_symbol):
    if i == j:
        return {ImmutableTree(start_symbol, [back_pointers[i][j][start_symbol][0]])}

    trees = set()

    for B, C, k in back_pointers[i][j][start_symbol]:
        left_trees = extract_trees(back_pointers, i, k, B)
        right_trees = extract_trees(back_pointers, k + 1, j, C)

        # combine the left and right subtrees
        for left in left_trees:
            for right in right_trees:
                trees.add(ImmutableTree(start_symbol, [left, right]))

    return trees

In [218]:
# test the CKY parser
for sentence in test_sentences[:2]:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    if success:
        trees = extract_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())
        print("Sentence: ", sentence[0], len(trees))
        for tree in trees:
            tree = nltk.tree.Tree.fromstring(str(tree))
            tree.pretty_print()
            tree.draw()
    else:
        print("Parsing failed for sentence: ", sentence[0])

Sentence:  ['prices', '.'] 2
         SIGMA            
    _______|________       
VERB_VBZ       pt_char_per
   |                |      
 prices             .     

         SIGMA            
    _______|________       
NOUN_NNS       pt_char_per
   |                |      
 prices             .     

Sentence:  ['show', 'availability', '.'] 3
         SIGMA                    
   ________|________               
  |                JXI            
  |         ________|_______       
NP_NN   NOUN_NN        pt_char_per
  |        |                |      
 show availability          .     

           SIGMA                    
    _________|________               
   |                 JWB            
   |          ________|_______       
NOUN_NN   AVPNP_NN       pt_char_per
   |         |                |      
  show  availability          .     

           SIGMA                    
    _________|________               
   |                 JKA            
   |          ________|_____

In [219]:
# Output the parsing results and compare with standard results (running for about 2.5 minutes)
for sentence in test_sentences:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    try:
        tree_num = len(list(parser.parse(sentence[0])))
    except Exception as e:
        tree_num = 0

    trees = []
    if success:
        trees = extract_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())
        print(sentence[0], '\t', len(trees))
    else:
        print(sentence[0], '\t', 0)

    if len(trees) != tree_num:
        print("Error: ", sentence[0], len(trees), tree_num)
        exit()

print("All tests passed!")

['prices', '.'] 	 2
['show', 'availability', '.'] 	 3
['show', 'the', 'flights', '.'] 	 2
['milwaukee', 'to', 'detroit', '.'] 	 2
['indianapolis', 'to', 'seattle', '.'] 	 2
['list', 'round', 'trips', '.'] 	 11
['list', 'saturday', 'flights', '.'] 	 5
['what', 'aircraft', 'is', 'this', '.'] 	 0
['list', 'these', 'economy', 'fares', '.'] 	 0
['list', 'these', 'city', 'destinations', '.'] 	 0
['what', 'is', 'the', 'fare', '.'] 	 2
['which', 'flights', 'are', 'cheapest', '.'] 	 0
['list', 'flights', 'from', 'cleveland', '.'] 	 5
['what', 'are', 'the', 'costs', '.'] 	 4
['can', 'i', 'have', 'the', 'fare', '.'] 	 1
['what', 'is', 'e', 'w', 'r', '.'] 	 1
['oakland', 'to', 'salt', 'lake', 'city', '.'] 	 19
['show', 'me', 'northwest', 'flights', 'to', 'detroit', '.'] 	 17
['i', 'want', 'to', 'leave', 'before', 'noon', '.'] 	 1
['what', 'flights', 'leave', 'boston', 'to', 'pittsburgh', '.'] 	 3
['i', "'d", 'like', 'an', 'afternoon', 'flight', '.'] 	 9
['are', 'any', 'fares', 'cheaper', 'than', '

In [220]:
# Extra: Figure out how to compute the number of parse trees with backpointers
def count_trees(back_pointers, i, j, start_symbol):
    if i == j:
        return 1

    count = 0

    for B, C, k in back_pointers[i][j][start_symbol]:
        left_count = count_trees(back_pointers, i, k, B)
        right_count = count_trees(back_pointers, k + 1, j, C)

        # just multiply the left trees number and right trees number
        count += left_count * right_count

    return count

In [221]:
# Extra: test count_trees (running for about 2 minutes)
for sentence in test_sentences:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    try:
        tree_num = len(list(parser.parse(sentence[0])))
    except Exception as e:
        tree_num = 0

    tree_count = 0
    if success:
        tree_count = count_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())

    if tree_count != tree_num:
        print("Error: ", sentence[0], tree_count, tree_num)
        exit()

    print(sentence[0], '\t', tree_count)

print("All tests passed!")

['prices', '.'] 	 2
['show', 'availability', '.'] 	 3
['show', 'the', 'flights', '.'] 	 2
['milwaukee', 'to', 'detroit', '.'] 	 2
['indianapolis', 'to', 'seattle', '.'] 	 2
['list', 'round', 'trips', '.'] 	 11
['list', 'saturday', 'flights', '.'] 	 5
['what', 'aircraft', 'is', 'this', '.'] 	 0
['list', 'these', 'economy', 'fares', '.'] 	 0
['list', 'these', 'city', 'destinations', '.'] 	 0
['what', 'is', 'the', 'fare', '.'] 	 2
['which', 'flights', 'are', 'cheapest', '.'] 	 0
['list', 'flights', 'from', 'cleveland', '.'] 	 5
['what', 'are', 'the', 'costs', '.'] 	 4
['can', 'i', 'have', 'the', 'fare', '.'] 	 1
['what', 'is', 'e', 'w', 'r', '.'] 	 1
['oakland', 'to', 'salt', 'lake', 'city', '.'] 	 19
['show', 'me', 'northwest', 'flights', 'to', 'detroit', '.'] 	 17
['i', 'want', 'to', 'leave', 'before', 'noon', '.'] 	 1
['what', 'flights', 'leave', 'boston', 'to', 'pittsburgh', '.'] 	 3
['i', "'d", 'like', 'an', 'afternoon', 'flight', '.'] 	 9
['are', 'any', 'fares', 'cheaper', 'than', '

In [222]:
# Extra: compare the efficiency improvement (run time: 15s)
import time

t1 = time.time()

for sentence in test_sentences:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    if success:
        trees = extract_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())

t2 = time.time()

for sentence in test_sentences:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    if success:
        tree_num = count_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())

t3 = time.time()

print("Time cost improvement: ", ((t2 - t1) - (t3 - t2)) / (t2 - t1))

Time cost improvement:  0.24149973829250632


In [223]:
sentence = test_sentences[12]
success, back_pointers = CKY_parse(sentence[0], grammar)
if success:
    trees = extract_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())
    print("Sentence: ", sentence[0], len(trees))
    for tree in trees:
        tree = nltk.tree.Tree.fromstring(str(tree))
        tree.pretty_print()
        tree.draw()

Sentence:  ['list', 'flights', 'from', 'cleveland', '.'] 5
                SIGMA                             
   _______________|______                          
  |                     GRY                       
  |       _______________|_______                  
  |      |                      GRZ               
  |      |                _______|__________       
  |      |             PP_NP                |     
  |      |         ______|_______           |      
NP_NN VERB_VBZ PREP_IN        NOUN_NP  pt_char_per
  |      |        |              |          |      
 list flights    from        cleveland      .     

                         SIGMA                               
    _______________________|_____                             
   |                            GVY                          
   |                _____________|_____________               
   |             NP_NNS                       GVZ            
   |        _______|_______              ______|_______       

# Tree figures
![Tree1](./trees/tree1.png "Tree1")
![Tree2](./trees/tree1.png "Tree2")
![Tree3](./trees/tree1.png "Tree3")
![Tree4](./trees/tree1.png "Tree4")
![Tree5](./trees/tree1.png "Tree5")